# Hello, _nbpresent_!

In [14]:
import nbpresent
nbpresent.__version__

'3.0.2'

In [15]:
import time

import bokeh.plotting
bokeh.plotting.output_notebook()
now=time.gmtime()
print("This notebook was last updated: "+time.asctime(now)+" UTC")

Loading BokehJS ...

This notebook was last updated: Fri Sep 16 02:39:05 2016 UTC


In [16]:
import astropy.io.votable as votab
import astropy.io.ascii 


import numpy as np

#download latest target lists:

latest_RV_url = "https://raw.githubusercontent.com/dsavransky/EXOSIMS/master/EXOSIMS/PlanetPopulation/RVplanets_ipac_2016-05-15.votable"
exoarch_confirmed_url = "http://exoplanetarchive.ipac.caltech.edu/cgi-bin/nstedAPI/nph-nstedAPI?table=exoplanets&format=votable"
exoplanets_eu = "http://www.exoplanet.eu/catalog/votable/" #times out
EXOSIM_stars = "https://raw.githubusercontent.com/dsavransky/EXOSIMS/master/EXOSIMS/StarCatalog/mission_exocat.votable"
oec_latest_tab = "https://raw.githubusercontent.com/douglase/oec_tables/master/comma_separated/open_exoplanet_catalogue.txt"

In [17]:
#parse
exocat = votab.parse(EXOSIM_stars,).get_first_table().to_table()
#knownRV = votab.parse("../EXOSIMS/EXOSIMS/PlanetPopulation/RVplanets_ipac_032216.votable").get_first_table().to_table()
knownRV = votab.parse(latest_RV_url).get_first_table().to_table()
exoarchConfirmed = votab.parse(exoarch_confirmed_url).get_first_table().to_table()
#exoplanets_dot_eu = votab.parse(exoplanets_eu).get_first_table().to_table()
#create 

In [18]:
#Define open exoplanet catalog column names:
oec_cols=["name",#  1: Primary identifier of planet
          "binary",#  2: Binary flag [0=no known stellar binary companion; 1=P-type binary (circumbinary); 2=S-type binary; 3=orphan planet (no star)]
          "pl_massj",##  3: Planetary mass [Jupiter masses]
          "pl_radiusj",##  4: Radius [Jupiter radii]
          "pl_orbper",#  5: Period [days]
          "pl_orbsmax",#  6: Semi-major axis [Astronomical Units]
          "pl_orbeccen",#  7: Eccentricity
          "pl_orblper",#  8: Periastron [degree]
          "longitude", ##  9: Longitude [degree] https://github.com/OpenExoplanetCatalogue/open_exoplanet_catalogue/issues/292
          "ascendingnode",#"10: Ascending node [degree]",
          "pl_orbincl",#"# 11: Inclination [degree]
          "pl_eqt",# 12: Surface or equilibrium temperature [K]
          "pl_age",# 13: Age [Gyr]
          "pl_discmethod",#,14: Discovery method
          "pl_disc",# 15:" Discovery year [yyyy]
          "rowupdate",# 16: Last updated [yy/mm/dd]
          "ra_str",# 17: Right ascension [hh mm ss]
          "dec_str",# 18: Declination [+/-dd mm ss]
          "st_dist",# 19: Distance from Sun [parsec]
          "st_mass",# 20: Host star mass [Solar masses]
          "st_rad",# 21: Host star radius [Solar radii]
          "st_metfe",# 22: Host star metallicity [log relative to solar]
          "st_teff",# 23: Host star temperature [K],"24: Host star age [Gyr]"]
          "st_age",# 24: Host star age [Gyr]
          "list",#detection status list
]
          
oec=astropy.io.ascii.read(oec_latest_tab,data_start=31,names=oec_cols,delimiter=",",guess=False)
oec=oec[oec["list"]=="Confirmed planets"]

In [19]:
def scatter_with_labels(p,x,y,names,**kwargs):
    #data source for mouseover
    if isinstance(names[0],bytes):
        names= [j.decode('utf-8') for j in names]
    source = bokeh.models.ColumnDataSource(data=dict(
        x=x,
        y=y,
        name=names, #convert bytes to strings
    ))

    p.scatter(x,y,line_color=None,source=source,**kwargs)
tooltips=[("index", "$index"),
          ("(x,y)", "(@x, @y)"),
          ("name", "@name")]

In [20]:

TOOLS="hover,crosshair,pan,wheel_zoom,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select"

p = bokeh.plotting.figure(tools=TOOLS, x_range=( 10000,0))

p.scatter(exocat['st_teff'], (exocat['st_mass']), radius=50,
          fill_color='gray', fill_alpha=0.5,legend="ExoCat",
          line_color=None)

p.scatter(knownRV['st_teff'], (knownRV["st_mass"]), radius=100,
fill_color="orange", fill_alpha=0.2,legend="Known RV",
          line_color=None)
p.xaxis.axis_label = "T [K]"
p.yaxis.axis_label = "Mass [M_sun]"
bokeh.plotting.show(p)


In [21]:

hover = bokeh.models.HoverTool(tooltips=tooltips)
tools=[hover]+["crosshair,pan,wheel_zoom,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select"]
p = bokeh.plotting.figure(tools=tools,y_axis_type='log', y_range=(1e-3,1e3) ,x_range=( 10000,0))

scatter_with_labels(p,exocat['st_teff'],exocat['st_lbol'],
                    exocat["hip_name"],
                    color="blue",legend="exocat",size=10,fill_alpha=0.1)

scatter_with_labels(p,knownRV["st_teff"],10**(knownRV["st_lum"]), 
                    knownRV["pl_name"],
                    color="orange",legend="Known RV",size=10,fill_alpha=.3)

p.title.text = "H-R Diagram"
p.xaxis.axis_label = "T [K]"
p.yaxis.axis_label = "Luminousity [L_sun]"
bokeh.plotting.show(p)


In [22]:

hover = bokeh.models.HoverTool(tooltips=tooltips)
tools=[hover]+["crosshair,pan,wheel_zoom,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select"]
p = bokeh.plotting.figure(tools=tools,y_axis_type='log', y_range=(1,1e3) ,x_range=( 10000,0))


p.scatter(exocat['st_teff'], (exocat['st_dist']), 
                              exocat["hip_name"],
          size=5,
          fill_color='gray', fill_alpha=0.2,legend="ExoCat",
          line_color=None)

p.scatter(knownRV['st_teff'], (knownRV["st_dist"]),
                              knownRV["pl_name"], size=10,
          fill_color="blue", fill_alpha=0.2,legend="Known RV",
          line_color=None)

p.scatter(exoarchConfirmed['st_teff'], (exoarchConfirmed["st_dist"]), size=10,
fill_color="green", fill_alpha=0.1,legend="IPAC Exoplanet Archive",
          line_color=None)
p.xaxis.axis_label = "T [K]"
p.yaxis.axis_label = "distance [pc]"
bokeh.plotting.show(p)




In [23]:

hover = bokeh.models.HoverTool(tooltips=tooltips)
tools=[hover]+["crosshair,pan,wheel_zoom,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select"]
p = bokeh.plotting.figure(tools=tools,y_axis_type='log', y_range=(1,1e3) ,x_range=( 10000,0))

scatter_with_labels(p,exocat['st_teff'], (exocat['st_dist']),
                    exocat["hip_name"],
                    radius=50,
          fill_color='gray', fill_alpha=0.2,legend="ExoCat",)
scatter_with_labels(p,knownRV['st_teff'], (knownRV["st_dist"]), knownRV["pl_name"],
                    radius=100,
                    fill_color="blue", fill_alpha=0.2,legend="Known RV",)
scatter_with_labels(p,exoarchConfirmed['st_teff'], (exoarchConfirmed["st_dist"]), exoarchConfirmed["pl_hostname"],
                    radius=100,
                    fill_color="green", fill_alpha=0.2,legend="IPAC Exoplanet Archive",)
p.xaxis.axis_label = "T [K]"
p.yaxis.axis_label = "distance [pc]"
bokeh.plotting.show(p)



In [29]:
hover = bokeh.models.HoverTool(tooltips=tooltips)


tools=[hover]+["crosshair,pan,wheel_zoom,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select"]

p = bokeh.plotting.figure(tools=tools,x_axis_type="log",  y_range=(-5, 35))#x_range=(0, 10000),  )

scatter_with_labels(p,exoarchConfirmed["pl_orbper"],exoarchConfirmed["st_dist"],
                    exoarchConfirmed["pl_hostname"],
                    color="gray",legend="IPAC Exoplanet Archive",size=6,fill_alpha=0.6)

scatter_with_labels(p,knownRV["pl_orbper"],knownRV["st_dist"], 
                    knownRV["pl_hostname"],
                    color="blue",legend="Known RV",size=10,fill_alpha=.35)

scatter_with_labels(p,oec["pl_orbper"],oec["st_dist"], 
                    oec["name"],
                    color="orange",legend="Open Exoplanet Catalog Confirmed",size=10,fill_alpha=.35)
p.xaxis.axis_label = "orbital period [d]"
p.yaxis.axis_label = "distance [pc]"



bokeh.plotting.show(p)



In [28]:
hover = bokeh.models.HoverTool(tooltips=tooltips)
tools=[hover]+["crosshair,pan,wheel_zoom,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select"]

p = bokeh.plotting.figure(tools=tools,x_axis_type="log",  y_range=(-5, 35),x_range=(1e-3, 1000),  )

scatter_with_labels(p,exoarchConfirmed["pl_bmassj"],exoarchConfirmed["st_dist"],
                    exoarchConfirmed["pl_hostname"],        
                    size=12,fill_alpha=0.34,
                    color="blue",legend="IPAC Exoplanet Archive")

scatter_with_labels(p,knownRV["pl_msinij"],knownRV["st_dist"], 
                    knownRV["pl_hostname"],size=12,fill_alpha=0.34,
                    color="green",legend="Known RV")

scatter_with_labels(p,oec["pl_massj"],oec["st_dist"], 
                    oec["name"],
                    color="orange",legend="Open Exoplanet Catalog Confirmed Planets",size=10,fill_alpha=.35)
p.xaxis.axis_label = "orbital period [d]"
scatter_with_labels(p,oec["pl_massj"],oec["st_dist"],oec["name"],
                    size=10,color='orange',fill_alpha=.3)
p.xaxis.axis_label = "msini [Jupiter]"
p.yaxis.axis_label = "distance [pc]"



bokeh.plotting.show(p)



In [26]:
hover = bokeh.models.HoverTool(tooltips=tooltips)
tools=[hover]+["crosshair,pan,wheel_zoom,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select"]

p = bokeh.plotting.figure(tools=tools,x_axis_type="log",  y_range=(-5, 35),x_range=(0, 25),  )

scatter_with_labels(p,exoarchConfirmed["st_optmag"],exoarchConfirmed["st_dist"],
                    exoarchConfirmed["pl_hostname"],
                    color="green",legend="IPAC Exoplanet Archive",
                    size=12,fill_alpha=0.34)

scatter_with_labels(p,knownRV["st_optmag"],knownRV["st_dist"], 
                    knownRV["pl_hostname"],
                    size=12,fill_alpha=0.34,
                    color="blue",legend="Known RV")

bokeh.plotting.show(p)
p.xaxis.axis_label = "Optical Mag"
p.yaxis.axis_label = "distance [pc]"

#what is optical mag? Johnson v_mag?


